# 12장 다중 분류 문제 해결하기
## 1. 다중 분류 문제
아리리스는 그 꽃봉오리가 마치 먹물을 머금은 붓과 같다 하여 우리나라에서는 '붓꽃'이라고 불리는 아름다운 꽃입니다. 아이리스는 꽃잎의 모양과 기링에 따라 여러 가지 품종으로 나뉩니다. 사진을 보면 서로 비슷해 보입니다. 과연 딥러닝을 사용하여 이들을 구별해 낼 수 있을까요?
<br><center>
(그림. 12-1) 아이리스의 품종<br>
<img src="https://drive.google.com/uc?id=10IA-RltFVLEn2BSsPRfR5nnDsRcqJJCe"><br>
<img src="https://drive.google.com/uc?id=1U8SR_6E9uuVO0vodKIg3l1zEOd2giP-z">
</center><br>  

아이리스 품종 예측 데이터는 예제 파일의 data 폴더에서 찾을 수 있습니다(data/iris3.csv). 데이터 구조는 다음과 같습니다.  
<br><center>
(그림. 12-2) 아이리스 데이터의 샘플, 속성, 클래스 구분<br>
<img src="https://drive.google.com/uc?id=1EEgYhS0DXYRtkKb5NXRzQXrosazQpZmq">
</center><br>  

* 샘플 수 : 150
* 속성의 수
 - 정보 1 : 꽃받침 길이(sepal length, 단위: cm)
 - 정보 2 : 꽃받침 너비(sepal width, 단위: cm)
 - 정보 3 : 꽃잎 길이(petal length, 단위: cm)
 - 정보 4 : 꽃잎 너비(petal width, 단위: cm)
* 클래스: iris-setosa, Iris-versicolor, Iris-virginica

속성을 보니 우리가 앞서 다루었던 데이터셋과 눈에 띠는 차이가 있습니다. 바로 클래스가 두 개가 아니라 세 개입니다. 즉, 참(1)과 거짓(0)을 해결하는 것이 아니라, 여러 개 중에 어떤 것이 답인지 예측하는 문제입니다. 

이렇게 여러 개의 중에 하나로 분류하는 것을 **다중 분류**(multi classification)라고 합니다. 다중 분류 문제는 둘 중에 하나를 고르는 이항 분류(binary classification)와는 접근 방식이 조금 다릅니다. 지금부터 아이리스 품종을 예측하는 실습을 통해 다중 분류 문제를 해결해 보겠습니다. 



### 2. 상관도 그래프

먼저 데이터를 불러와 데이터셋의 일부를 확인하겠습니다.

데이터 프레임에 들어 있는 데이터셋(샘플들)은 같은 품종끼리 정렬이 되어 있습니다. 품종(species)에 따른 샘플이 몇 개씩 있는지 확인해 보겠습니다.

데이터셋에 setosa, versicolor, virginica 각각 50개 샘플이 저장되어 있는 것을 확인했습니다. 이제 상관도 그래프를 그려보겠습니다. 시본(seaborn) 라이브러리에 있는 pairplot() 함수를 써서 전체 상관도를 제시합니다.

위 상관도를 통해 각 속성별 데이터 분포와 속성 간의 관계를 한눈에 볼 수 있습니다. pairplot() 함수 설정 중 hue 옵션은 주어진 데이터 중 어떤 카테고리를 중심으로 그래프를 그릴지 정해 주게 되는데, 우리는 품종(species)에 따라 보여지게끔 지정했습니다.
```
sns.pairplot(df, hue='species')
```
그래프 각가의 가로추과 세로축은 서로 다른 속성을 나타내며, 이러한 속성에 따라 품종이 어떻게 분포하는지 보여 줍니다. 이렇한 분석을 통해 시진 상으로 비슷해 보이던 꽃잎과 꽃받침의 크기와 너비가 품종별로 어떤 차이가 있는지 알 수 있습니다. 

예를 들어, 위 그림의 맨 왼쪽 맨 아래 그래프를 보겠습니다. 이 그래프에서 파란색 원형 점과 주황색 원형 점 그리고 녹생 원형 점이 표시되어 세 품종에 대한 정보를 담고 있습니다. 파란색은 원형 점은 setosa 품종에 대한 정보를, 주황색 점들은 versicolor 품종에 대한 정보를 그리고 녹색 점들은 viriginica 품종에 대한 정보를 보이고 있습니다. 파란색 점들은 setosa 품종의 분꽃들의 꽃받침(sepal)의 길이와 꽃잎(petal)의 너비와의 상관관계를 보이고 있습니다. 꽃받침(sepal)의 길이 변화 대비 꽃잎(petal)의 너비 변화가 크지 않다고 판단됩니다. 

위 서브 그래프 중, setosa 붓꽃의 경우 꽃받침의 폭(sepal width) 변화 대비 꽃잎의 길이(petal length) 변화가 그리 크지 않다 또는 관계가 깊지 않다고 판단할 수 있는 정보를 주는 그래프는 어떤 것일까요?  

<br><center>
상과 관계가 매우 낮음(없음)<br>
<img src="https://drive.google.com/uc?id=1SCa732olmGpw-Nhfv0YC1mhbZyFS7xio">
</center><br>  



## 3. 원-핫 인코딩
이제 케라스를 이용해 아이리스의 품종을 예측해 보겠습니다. 먼저 데이터 프레임(df)를 X와 y로 나누겠습니다.

$y$의 값이 숫자가 아닌 문자입니다. 품종(species)에 대한 정보가 "Iris-setosa" 등 문자로 되어 있기 때문이네요. 원활한 계산을 위해 문자를 숫자형으로 바꾸어 표현하겠습니다.  

아이리스 꽃의 품종은 세 가지 입니다. 그러면 각각의 이름으로 세 개의 열을 만든 후 품종이 일치하는 경우 1로 아닌 경우는 0으로 변경합니다.
<br><center>
원-핫 인코딩<br>
<img src="https://drive.google.com/uc?id=1x3dOjXtJG7ZquDfVlEXT27k_EjkoNMy2" width=500>
</center><br>  

이렇게 여러 개의 값으로 된 문자열을 0과 1로만 이루어진 형태로 만들어 주는 과정을 **원-핫 인코딩**(one-hot encoding)이라고 합니다. 원-핫 인코딩은 판다스가 제공하는 get_dummies() 함수를 사용하면 간단하게 만들 수 있습니다. 


## 4. 소프트맥스

이제 모델을 만들어 줄 차례입니다. 다음 코드를 보면서 이전에 실행했던 피마 인디언의 당뇨병 예측과 무엇이 달라졌는지 찾아 보세요.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# 모델 설정
model = Sequential()
model.add(Dense(12,  input_dim=4, activation='relu'))
model.add(Dense(8,  activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X, y, epochs=30, batch_size=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 12)                60        
                                                                 
 dense_1 (Dense)             (None, 8)                 104       
                                                                 
 dense_2 (Dense)             (None, 3)                 27        
                                                                 
Total params: 191
Trainable params: 191
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
30/30 [==============================] - 1s 2ms/step - loss: 1.6544 - accuracy: 0.4267
Epoch 2/30
30/30 [==============================] - 0s 7ms/step - loss: 1.2407 - accuracy: 0.6667
Epoch 3/30
30/30 [==============================] - 0s 3ms/step - loss: 0.9688 - accuracy: 0.6667
Epoch 4/30
30/30 [==========

적확도가 무려 약 97%가 나왔네요. 이전에 실행했던 피마 인디언의 당뇨병 예측과 무엇이 달라졌는지 찾아 보셨습니까? 세 가지가 달라졌습니다. 출력 층의 노드 수가 3으로 변경되었습니다. 그리고 활성화 함수가 softmax로 변경되었습니다. 또 컴파일 부분에서 손실 함수가 categorical_crossentropy로 바뀌었습니다. 

먼저 출력 부분에 대해 알아보겠습니다. 이전까지 우리는 출력이 0\~1 사이의 한 값으로 나왔습니다. 예를 들어 당뇨인지 아니지에 대한 예측 값이 시그로임드 함수를 거쳐 0\~1 사이의 값 중 하나로 변환되어 0.5 이상이면 당뇨로, 이하면 정상으로 판단했지요. 그런데 이번 예제에서는 예측해야할 값이 세 가지 늘었습니다. 즉, 각 샘플마다 이것이 setosa일 확율, versicolor일 확율 그리고 virginica일 확율을 따로따로 구해야 한다는 것이지요. 예를 들어 예측 결과는 그림(12-5)또는 표 (12-1)와 같은 형태로 나타납니다.  

<br><center>
(그림 12-5) 소프트 맥스<br>
<img src = "https://drive.google.com/uc?id=1cfHTDo5ZnsxIODeYyS3nHv3NQtSxxhp-"><br>

(표. 12-1) 소프트맥스<br>  

|setona일 확율|versicolor일 확율|virginca일 확율|
|---|---|---|
|0.2|0.7|0.1|
|0.8|0.1|0.1|
|0.2|02.|0.6|

</center><br>

이렇게 세 가지의 확율을 모두 구해야 하므로 시그모이드 함수가 아닌 다른 함수가 필요합니다. 이때 사용되는 함수가 바로 소프트맥스 함수입니다. 소프트 맥스 함수는 표(12-1)과 같이 각 항목당 예측 확율이 0과 1 사이의 값으로 나타내 주는데, 이때 각 샘플당 예측 확율의 총합이 1인 형태로 바꾸어 주게 됩니다. 

마찬가지로 손실 함수도 이전과 달라졌습니다. 이항 분류에서 binary_crossentropy를 썼다면, 다항 분류에서는 categorical_crossentropy를 쓰면 됩니다. 

## 5. 아이리스 품족 예측 실행

```model.summary()```를 사용해서 두 개의 은닉층에 각각 12개와 여덟 개의 노드가 만들어졌고 출력은 세 개임을 확인할 수 있습니다. 결과는 30번 반복했을 정확도가 약 96%(실행 시킬 때 마다 차이가 발생할 수 있음) 나왔습니다. 꽃의 너비와 길이를 담은 150개의 데이터 중 144개의 꽃 종류를 맞추었다는 의미입니다. 이제부터는 이렇게 측정된 정확도를 어떻게 신뢰할 수 있을지, 예측 결과의 신뢰도를 높이는 방법에 대해 알아보겠습니다. 